In [18]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import classifier as cl
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()


seed = 1
dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed)

x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]
y_train, y_test = dataset_orig_train.labels.reshape((-1,)), dataset_orig_test.labels.reshape((-1,))





## Running linear regression to get sensetive directions 

protected_regression = linear_model.LinearRegression(fit_intercept = False)
protected_regression.fit(x_unprotected_train, x_protected_train)
sensetive_directions = protected_regression.coef_







unprotected_directions = utils.projection_matrix(sensetive_directions)



# Casing to tensor 
y_train, y_test = y_train.astype('int32'), y_test.astype('int32')
x_unprotected_train, x_unprotected_test = tf.cast(x_unprotected_train, dtype = tf.float32), tf.cast(x_unprotected_test, dtype = tf.float32)
y_train, y_test = tf.one_hot(y_train, 2), tf.one_hot(y_test, 2)
unprotected_directions = tf.cast(unprotected_directions, dtype = tf.float32)

init_graph = utils.ClassifierGraph(50, 2)
#graph = cl.Classifier(init_graph, x_unprotected_train, y_train, x_unprotected_test, y_test, num_steps = 10000) # use for unfair algo
graph = cl.Classifier(init_graph, tf.matmul(x_unprotected_train, unprotected_directions), \
                        y_train, tf.matmul(x_unprotected_test, unprotected_directions), y_test, num_steps = 10000) # for fair algo


def fair_transform(x):
    return tf.matmul(x, unprotected_directions)

Done step 200

Done step 400

Done step 600

Done step 800

Done step 1000

Done step 1200

Done step 1400

Done step 1600

Done step 1800

Done step 2000

Done step 2200

Done step 2400

Done step 2600

Done step 2800

Done step 3000

Done step 3200

Done step 3400

Done step 3600

Done step 3800

Done step 4000

Done step 4200

Done step 4400

Done step 4600

Done step 4800

Done step 5000

Done step 5200

Done step 5400

Done step 5600

Done step 5800

Done step 6000

Done step 6200

Done step 6400

Done step 6600

Done step 6800

Done step 7000

Done step 7200

Done step 7400

Done step 7600

Done step 7800

Done step 8000

Done step 8200

Done step 8400

Done step 8600

Done step 8800

Done step 9000

Done step 9200

Done step 9400

Done step 9600

Done step 9800

Done step 10000



In [19]:
x = x_unprotected_test[1]
y = y_test[1]

In [20]:
learning_rate = 1e-3
num_steps = 200

In [21]:
x = tf.reshape(x, (1, -1))
y = tf.reshape(y, (1, -1))
x_start = x
x_base = x
x_fair = x
for _ in range(num_steps):
    with tf.GradientTape() as g:
        g.watch(x_base)
        prob_base = graph(fair_transform(x_base))
        prob_start = graph(fair_transform(x_start))
        perturb = x_base - x_start
        loss = (utils.kl(prob_base, prob_start))/(tf.norm(perturb)+1)

    gradient = g.gradient(loss, x_base)
    x_base = x_base + learning_rate * gradient 
    print(gradient)

    
for _ in range(num_steps):
    with tf.GradientTape() as g:
        g.watch(x_fair)
        prob_fair = graph(fair_transform(x_fair))
        prob_start = graph(fair_transform(x_start))
        perturb = tf.matmul(x_fair - x_start, unprotected_directions)
        loss = (utils.kl(prob_fair, prob_start))/(tf.norm(perturb)+1)

    gradient = g.gradient(loss, x_fair)
    x_fair = x_fair + learning_rate * gradient 

.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape

In [5]:
x_base

<tf.Tensor: shape=(1, 39), dtype=float32, numpy=
array([[ 0.26477048, -0.43578398, -0.14741328, -0.219069  , -0.0756966 ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]],
      dtype=float32)>

In [6]:
x_fair

<tf.Tensor: shape=(1, 39), dtype=float32, numpy=
array([[ 0.26477048, -0.43578398, -0.14741328, -0.219069  , -0.0756966 ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]],
      dtype=float32)>

In [7]:
prob_fair = graph(fair_transform(x_fair))
prob_base = graph(fair_transform(x_base))
prob_start = graph(fair_transform(x_start))

lf_fair = utils.kl(prob_fair, prob_start)/ (tf.norm(tf.matmul(x_fair-x_start, unprotected_directions)) + 1)
lf_base = utils.kl(prob_base, prob_start)/ (tf.norm(tf.matmul(x_base-x_start, unprotected_directions)) + 1)
lb_fair = utils.kl(prob_fair, prob_start)/ (tf.norm(x_fair-x_start) + 1)
lb_base = utils.kl(prob_base, prob_start)/ (tf.norm(x_base-x_start) + 1)
lf_fair, lf_base, lb_fair, lb_base

(<tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0>)

In [14]:
tf.norm(tf.matmul(x_fair-x_start, unprotected_directions))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [15]:
tf.norm(tf.matmul(x_base-x_start, unprotected_directions))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [16]:
tf.norm(x_fair-x_start)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [17]:
tf.norm(x_base-x_start)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [41]:
utils.kl(prob_fair, prob_start)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [42]:
utils.kl(prob_base, prob_start)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [2]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import classifier as cl
import utils
import random
import matplotlib.pyplot as plt
import scipy


seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)


expt = '_1'
filename = f'output/test_distance_ratios{expt}.npy'
histplot = f'output/perturbed-mean-entropy-hist{expt}.png'
qqplot = f'output/perturbed-mean-entropy-qqplot{expt}.png'


test_ratio =  np.load(filename)




def ratio_mean(n = 9045):
    index = random.sample(range(n), 1000)
    srswr_ratio=[test_ratio[i] for i in index]
    return np.mean(srswr_ratio)

ratio_means = [ratio_mean() for _ in range(5000)]
plt.hist(ratio_means)
plt.title(f'Histogram of mean loss of ratios for expt{expt}')
plt.xticks(rotation = 35, fontsize = 'x-small')
plt.savefig(histplot)

ValueError: autodetected range of [nan, nan] is not finite

In [3]:
test_ratio

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [22]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import classifier as cl
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()


seed = 1
dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed)

x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]
y_train, y_test = dataset_orig_train.labels.reshape((-1,)), dataset_orig_test.labels.reshape((-1,))





## Running linear regression to get sensetive directions 

protected_regression = linear_model.LinearRegression(fit_intercept = False)
protected_regression.fit(x_unprotected_train, x_protected_train)
sensetive_directions = protected_regression.coef_







unprotected_directions = utils.projection_matrix(sensetive_directions)



# Casing to tensor 
y_train, y_test = y_train.astype('int32'), y_test.astype('int32')
x_unprotected_train, x_unprotected_test = tf.cast(x_unprotected_train, dtype = tf.float32), tf.cast(x_unprotected_test, dtype = tf.float32)
y_train, y_test = tf.one_hot(y_train, 2), tf.one_hot(y_test, 2)
unprotected_directions = tf.cast(unprotected_directions, dtype = tf.float32)

init_graph = utils.ClassifierGraph(50, 2)
graph = cl.Classifier(init_graph, x_unprotected_train, y_train, x_unprotected_test, y_test, num_steps = 10000) # use for unfair algo
#graph = cl.Classifier(init_graph, tf.matmul(x_unprotected_train, unprotected_directions), \
#                        y_train, tf.matmul(x_unprotected_test, unprotected_directions), y_test, num_steps = 10000) # for fair algo


def fair_transform(x):
    return tf.matmul(x, unprotected_directions)

Done step 200

Done step 400

Done step 600

Done step 800

Done step 1000

Done step 1200

Done step 1400

Done step 1600

Done step 1800

Done step 2000

Done step 2200

Done step 2400

Done step 2600

Done step 2800

Done step 3000

Done step 3200

Done step 3400

Done step 3600

Done step 3800

Done step 4000

Done step 4200

Done step 4400

Done step 4600

Done step 4800

Done step 5000

Done step 5200

Done step 5400

Done step 5600

Done step 5800

Done step 6000

Done step 6200

Done step 6400

Done step 6600

Done step 6800

Done step 7000

Done step 7200

Done step 7400

Done step 7600

Done step 7800

Done step 8000

Done step 8200

Done step 8400

Done step 8600

Done step 8800

Done step 9000

Done step 9200

Done step 9400

Done step 9600

Done step 9800

Done step 10000



In [23]:
x = tf.reshape(x, (1, -1))
y = tf.reshape(y, (1, -1))
x_start = x
x_base = x
x_fair = x
for _ in range(num_steps):
    with tf.GradientTape() as g:
        g.watch(x_base)
        prob_base = graph((x_base))
        prob_start = graph((x_start))
        perturb = x_base - x_start
        loss = (utils.kl(prob_base, prob_start))/(tf.norm(perturb)+1)

    gradient = g.gradient(loss, x_base)
    x_base = x_base + learning_rate * gradient 
    print(gradient)

    
for _ in range(num_steps):
    with tf.GradientTape() as g:
        g.watch(x_fair)
        prob_fair = graph((x_fair))
        prob_start = graph((x_start))
        perturb = tf.matmul(x_fair - x_start, unprotected_directions)
        loss = (utils.kl(prob_fair, prob_start))/(tf.norm(perturb)+1)

    gradient = g.gradient(loss, x_fair)
    x_fair = x_fair + learning_rate * gradient 

.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 39), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape

In [24]:
x = tf.cast(1, dtype = tf.float32)

In [25]:
y = x

In [26]:
y = y+1

In [27]:
x

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [28]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>